In [133]:
import numpy as np
import pandas as pd
from pandas import DataFrame
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
import requests
from pandas.io.json import json_normalize
#!conda install -c conda-forge folium=0.5.0 --yes
import folium


print('Completed')

Completed


In [134]:
client_id = 'IWHVMX4LERJWS0TVZAQKIMNKSV112LMFATGEDOGMB0IBI0UH'
client_secret = '2PEAYMQS2ZUEOCLWZ1CMUEYL3SVUACGIMYV5S1MKVAORORUN'
version = '20180604'
limit = 100
print('Connected to FourSquare')

#add cities to lookup
cities = ['New York, NY','San Francisco, CA', 'Baltimore, MD', 'Boston, MA', 'Philadelphia, PA', 'New Orleans, LA', 'Seattle, WA', 'Los Angeles, CA', 'Chicago, IL', 'Washington DC']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        client_id, 
        client_secret, 
        version, 
        city,
        limit,
        "4bf58dd8d48988d181941735") # Museum category
    results[city] = requests.get(url).json()

Connected to FourSquare


In [135]:
#create dataframe for Museums
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [136]:
df_venues

{'New York, NY':                                                  Name  \
 0   The Metropolitan Museum of Art (Metropolitan M...   
 1                                   The Jewish Museum   
 2                       MoMA: Architecture and Design   
 3                                       Japan Society   
 4                      Whitney Museum of American Art   
 5                         Museum of Modern Art (MoMA)   
 6                  American Museum of Natural History   
 7                         The Morgan Library & Museum   
 8                                The Frick Collection   
 9                                           Sotheby's   
 10                                Rubin Museum of Art   
 11            Cooper Hewitt Smithsonian Design Museum   
 12                             9/11 Museum Auditorium   
 13                   Intrepid Sea, Air & Space Museum   
 14                    Rose Center for Earth and Space   
 15                                     The Met Breuer  

In [137]:
#create dataframe for cities researching
cities_df = DataFrame(cities, columns = ['City'])
cities_df

,City
0,"New York, NY"
1,"San Francisco, CA"
2,"Baltimore, MD"
3,"Boston, MA"
4,"Philadelphia, PA"
5,"New Orleans, LA"
6,"Seattle, WA"
7,"Los Angeles, CA"
8,"Chicago, IL"
9,Washington DC


In [138]:
#all of the information from foursquare
results

{'New York, NY': {'meta': {'code': 200,
   'requestId': '5f3ead863a36067f05d8eb95'},
  'response': {'suggestedFilters': {'header': 'Tap to show:',
    'filters': [{'name': 'Open now', 'key': 'openNow'},
     {'name': '$-$$$$', 'key': 'price'}]},
   'geocode': {'what': '',
    'where': 'new york ny',
    'center': {'lat': 40.742185, 'lng': -73.992602},
    'displayString': 'New York, NY, United States',
    'cc': 'US',
    'geometry': {'bounds': {'ne': {'lat': 40.882214, 'lng': -73.907},
      'sw': {'lat': 40.679548, 'lng': -74.047285}}},
    'slug': 'new-york-city-new-york',
    'longId': '72057594043056517'},
   'headerLocation': 'New York',
   'headerFullLocation': 'New York',
   'headerLocationGranularity': 'city',
   'query': 'museum',
   'totalResults': 189,
   'suggestedBounds': {'ne': {'lat': 40.814604422353845,
     'lng': -73.94259811972202},
    'sw': {'lat': 40.683589784903724, 'lng': -74.04980571937705}},
   'groups': [{'type': 'Recommended Places',
     'name': 'recommend

In [139]:
#creating maps for all cities
maps = {}
#creating a list of all mean distances
meandist = []
#creating a list of total museums
count = []
#creating a list for central location
central = []
for city in cities:
    city_lat = np.mean(results[city]['response']['geocode']['center']['lat'])
    city_lng = np.mean(results[city]['response']['geocode']['center']['lng'])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()]
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        # marker for museums
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
        #marker for central point
        label = folium.Popup("Center Point", parse_html=True)
        folium.CircleMarker(
            venues_coor,
            radius=5,
            popup=label,
            color='black',
            fill=True,
            fill_color='black',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])      
    #print information about each city    
    print(f"Total number of museums in {city} = ", results[city]['response']['totalResults'])
    if results[city]['response']['totalResults'] > 100:
        print("Showing Top 100")
        print("Mean Distance from Center Point is:")
        print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_coor),1,df_venues[city][['Lat','Lng']].values)))
    else: 
        print("Showing All ", results[city]['response']['totalResults'])
        print("Mean Distance from Center Point is:")
        print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_coor),1,df_venues[city][['Lat','Lng']].values)))
    
    #adding data to the lists created above
    meandist.append(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_coor),1,df_venues[city][['Lat','Lng']].values)))
        
    count.append(results[city]['response']['totalResults'])
    
    central.append(venues_coor)

Total number of museums in New York, NY =  189
Showing Top 100
Mean Distance from Center Point is:
0.028179112015207917
Total number of museums in San Francisco, CA =  119
Showing Top 100
Mean Distance from Center Point is:
0.036903794278582425
Total number of museums in Baltimore, MD =  52
Showing All  52
Mean Distance from Center Point is:
0.020073717409405126
Total number of museums in Boston, MA =  76
Showing All  76
Mean Distance from Center Point is:
0.025193997533364668
Total number of museums in Philadelphia, PA =  101
Showing Top 100
Mean Distance from Center Point is:
0.030391238804207542
Total number of museums in New Orleans, LA =  68
Showing All  68
Mean Distance from Center Point is:
0.01732314304627997
Total number of museums in Seattle, WA =  84
Showing All  84
Mean Distance from Center Point is:
0.042603402640903336
Total number of museums in Los Angeles, CA =  174
Showing Top 100
Mean Distance from Center Point is:
0.09572948648339497
Total number of museums in Chicag

In [140]:
#show New York
print(cities[0])
print(meandist[0])
maps[cities[0]]

New York, NY
0.028179112015207917


In [141]:
#show San Francisco
print(cities[1])
print(meandist[1])
maps[cities[1]]

San Francisco, CA
0.036903794278582425


In [142]:
#Show Baltimore
print(cities[2])
print(meandist[2])
maps[cities[2]]

Baltimore, MD
0.020073717409405126


In [143]:
#Show Boston
print(cities[3])
print(meandist[3])
maps[cities[3]]

Boston, MA
0.025193997533364668


In [144]:
#Show Philadelphia
print(cities[4])
print(meandist[4])
maps[cities[4]]

Philadelphia, PA
0.030391238804207542


In [145]:
#Show New Orleans
print(cities[5])
print(meandist[5])
maps[cities[5]]

New Orleans, LA
0.01732314304627997


In [146]:
#Show Seattle
print(cities[6])
print(meandist[6])
maps[cities[6]]

Seattle, WA
0.042603402640903336


In [147]:
#Show Los Angeles
print(cities[7])
print(meandist[7])
maps[cities[7]]

Los Angeles, CA
0.09572948648339497


In [148]:
#Show Chicago
print(cities[8])
print(meandist[8])
maps[cities[8]]

Chicago, IL
0.05576715324048896


In [149]:
#Show Washington DC
print(cities[9])
print(meandist[9])
maps[cities[9]]

Washington DC
0.01634594456940411


In [152]:
#Add Mean distance and total museums to dataframe
cities_df['Mean Distance'] = meandist
cities_df['Total Museums'] = count
cities_df['Max Museums Reviewed'] = np.where(cities_df['Total Museums']>100,100,cities_df['Total Museums'])
cities_df['Latitude, Longitude'] = central
cities_df = cities_df.set_index("City", drop = True)

In [153]:
#Show dataframe
cities_df

,Mean Distance,Total Museums,Max Museums Reviewed,"Latitude, Longitude"
City,,,,
"New York, NY",0.028179,189,100,"[40.75424234522059, -73.98281677705133]"
"San Francisco, CA",0.036904,119,100,"[37.78010728075259, -122.42588125502732]"
"Baltimore, MD",0.020074,52,52,"[39.29291402871929, -76.6160116179364]"
"Boston, MA",0.025194,76,76,"[42.35136614115616, -71.07083477066293]"
"Philadelphia, PA",0.030391,101,100,"[39.9593977887982, -75.16565891996316]"
"New Orleans, LA",0.017323,68,68,"[29.956932692429223, -90.06575947440294]"
"Seattle, WA",0.042603,84,84,"[47.60090287961491, -122.33114639130766]"
"Los Angeles, CA",0.095729,174,100,"[34.06576958566551, -118.3367902099242]"
"Chicago, IL",0.055767,146,100,"[41.86092322200809, -87.62846317781916]"


In [154]:
#connecting to foursquare and finding the closest hotel near the central location
client_id = 'IWHVMX4LERJWS0TVZAQKIMNKSV112LMFATGEDOGMB0IBI0UH'
client_secret = '2PEAYMQS2ZUEOCLWZ1CMUEYL3SVUACGIMYV5S1MKVAORORUN'
version = '20180604'
limit = 1
search_query = 'Hotel'
latlong = '38.89751372885385, -77.02835154630353'
radius = 500

#add LatLong to lookup
results = {}
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={}&v={}&query={}&radius={}&limit={}'.format(
        client_id, 
        client_secret, 
        latlong,
        version,
        search_query,
        radius,
        limit)
results = requests.get(url).json()


In [155]:
#closest hotel to central location of museums
results

{'meta': {'code': 200, 'requestId': '5f3eaef304da622edbfca5f6'},
 'response': {'venues': [{'id': '4b7dc686f964a520dfd32fe3',
    'name': 'Hotel Harrington',
    'location': {'address': '436 11th St NW',
     'lat': 38.89617549241759,
     'lng': -77.02792114612103,
     'labeledLatLngs': [{'label': 'display',
       'lat': 38.89617549241759,
       'lng': -77.02792114612103}],
     'distance': 153,
     'postalCode': '20004',
     'cc': 'US',
     'city': 'Washington',
     'state': 'D.C.',
     'country': 'United States',
     'formattedAddress': ['436 11th St NW',
      'Washington, D.C. 20004',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1fa931735',
      'name': 'Hotel',
      'pluralName': 'Hotels',
      'shortName': 'Hotel',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1597943538',
    'hasPerk': False}]}}

In [157]:
hotels = results['response']['venues']
dataframe = json_normalize(hotels)
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
#showing the closest hotel
dataframe_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Hotel Harrington,Hotel,436 11th St NW,US,Washington,United States,153,"[436 11th St NW, Washington, D.C. 20004, Unite...","[{'label': 'display', 'lat': 38.89617549241759...",38.896175,-77.027921,20004,D.C.,4b7dc686f964a520dfd32fe3
